In [1]:
import math
from interval import interval, imath
from Polynomials import *

In [2]:
class Taylor:
    def __init__(self, poly:Poly, remainder:interval, domain:interval):
        self.P=poly
        self.I=interval(remainder)
        self.D=interval(domain)

    @property
    def order(self):
        return self.P.order
    
    def __repr__(self):
        return 'TaylorModel({0}, {1}, {2})'.format(self.P, self.I, self.D)
    
    def __add__(self, other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,self.D)
        D=self.D&other.D
        return Taylor(self.P+other.P,self.I+other.I,D)
    
    def __radd__(self,other):
        return self+other
    
    def __sub__(self, other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,self.D)
        D=self.D&other.D
        return Taylor(self.P-other.P,self.I-other.I,D)
    
    def __rsub__(self,other):
        if isinstance(other,(int,float)):
            other=Taylor(Poly([other],0),0,self.D)
        return other-self
    
    # full multiplication, n is the result order, defualt n is 'full'
    def fullmul(self,other,n='full'):
        if isinstance(other,(int,float)):
            if other==0:
                return 0
            else:
                return Taylor(other*self.P,other*self.I,self.D)
        D=self.D&other.D
        P=self.P*other.P
        I1=self.P.bound(D)*other.I
        I2=self.I*other.P.bound(D)
        I=I1+I2+self.I*other.I
        if n=='full':
            return Taylor(P,I,D)
        elif isinstance(n,int) and n>0 and n<=(self.n+other.n):
            return Taylor(P,I,D).truncation(n)
        else:
            raise Exception("n must be 'full' or integer between 0 ~ self.order+other.order")
    
    def __mul__(self, other):
        if isinstance(other,(int,float)):
            if other==0:
                return 0
            else:
                return Taylor(other*self.P,other*self.I,self.D)
        return self.fullmul(other).truncation(max(self.order,other.order))
    
    def __rmul__(self,other):
        return self*other
    
    def bound(self):
        return self.P.bound(self.D)+self.I
    
    def __call__(self):
        return self.bound()
    
    def truncation(self,n):
        P=self.P.truncation(n)
        I=(self.P-P).bound(self.D)+self.I
        return Taylor(P,I,self.D)
    
    def __truediv__(self, other):
        if isinstance(other,(int,float)):
            if other==0:
                raise Exception("Cannot dividied by zero!")
            else:
                return Taylor(self.P/other,self.I/other,self.D)
        else:
            raise Exception("Can only do scalar division!")
    
    def one(self):
        return Taylor(Poly.one(self.order),self.I,self.D)
    
    def zero(self):
        return Taylor(Poly.zero(self.order),self.I,self.D)
    
    def square(self):
        return self*self
    
    def __pow__(self,n):
        if n==0:
            return self.one()
        elif n%2==0:
            if n==2:
                return self.square()
            else:
                p=int(n/2)
                return self.__pow__(p).square()
        elif n%2==1:
            if n==1:
                return self
            else:
                p=int((n-1)/2)
                return self*self.__pow__(p).square()
        else:
            raise Exception("n must be a positive integer")
    
    # 0~n coefficients of exp sin cos
    def coeff(f,n,x):   
        if f=='exp':
            return math.exp(x)/math.factorial(n)
        if f=='sin':
            i=n%4
            if i==0:
                return math.sin(x)/math.factorial(n)
            elif i==1:
                return math.cos(x)/math.factorial(n)
            elif i==2:
                return -math.sin(x)/math.factorial(n)
            elif i==3:
                return -math.cos(x)/math.factorial(n)
        if f=='cos':
            i=n%4
            if i==0:
                return math.cos(x)/math.factorial(n)
            elif i==1:
                return -math.sin(x)/math.factorial(n)
            elif i==2:
                return -math.cos(x)/math.factorial(n)
            elif i==3:
                return math.sin(x)/math.factorial(n)
    
    # n+1 coefficient 
    def coeff2(f,n,x):
        if f=='exp':
            return imath.exp(x)/math.factorial(n)
        if f=='sin':
            i=n%4
            if i==0:
                return imath.sin(x)/math.factorial(n)
            elif i==1:
                return imath.cos(x)/math.factorial(n)
            elif i==2:
                return -imath.sin(x)/math.factorial(n)
            elif i==3:
                return -imath.cos(x)/math.factorial(n)
        if f=='cos':
            i=n%4
            if i==0:
                return imath.cos(x)/math.factorial(n)
            elif i==1:
                return -imath.sin(x)/math.factorial(n)
            elif i==2:
                return -imath.cos(x)/math.factorial(n)
            elif i==3:
                return imath.sin(x)/math.factorial(n)
    
    # elementary functions
    def element(f,n,x0,D):
        coef=[]
        for i in range(n+1):
            coef.append(Taylor.coeff(f,i,x0))
        coef1=Taylor.coeff2(f,n+1,D)
        I=coef1*((D-x0)**(n+1))
        return Taylor(Poly(coef,n),I,D)
            
    #p(Taylor)
    def compo_poly(self,p,c='best'):
        if c=='naive' or c=='n':
            return p.bound_naive(self)
        elif c=='Horner' or c=='H':
            return p(self)
        elif c=='best':
            if p.bound_naive(self).I&p(self).I==p(self).I:
                return p(self)
            else: return p.bound_naive(self)
        else:
            raise Exception("The options mush be naive(n), Horner(H) or best")
            
    #composition of a function 'f' and a Taylor model 'self'
    def compose(self,f,n,x0):
        Bf=self.bound()
        Mg=Taylor.element(f,n,x0,Bf)
        M1=self.zero()
        M1.P.coef[1:]=self.P.coef[1:]
        M=M1.compo_poly(Mg.P,'best')
        return Taylor(M.P,M.I+Mg.I,self.D)

In [3]:
a=Taylor(Poly([0,1,2,3,4],4),interval([0,1]),interval([-1,1]))
b=Taylor(Poly([1,2,3,4,5],4),interval([0,2]),interval([-1,1]))
a

TaylorModel(Poly([0, 1, 2, 3, 4], 4), interval([0.0, 1.0]), interval([-1.0, 1.0]))

In [4]:
a.truncation(2)

TaylorModel(Poly([0, 1, 2], 2), interval([-3.0, 8.0]), interval([-1.0, 1.0]))

In [5]:
a.bound()

interval([-4.0, 11.0])

In [6]:
a()

interval([-4.0, 11.0])

In [7]:
a+b

TaylorModel(Poly([1, 3, 5, 7, 9], 4), interval([0.0, 3.0]), interval([-1.0, 1.0]))

In [8]:
1+a

TaylorModel(Poly([1, 1, 2, 3, 4], 4), interval([0.0, 1.0]), interval([-1.0, 1.0]))

In [9]:
a-a

TaylorModel(Poly([0, 0, 0, 0, 0], 4), interval([-1.0, 1.0]), interval([-1.0, 1.0]))

In [10]:
1-a

TaylorModel(Poly([1, -1, -2, -3, -4], 4), interval([-1.0, 0.0]), interval([-1.0, 1.0]))

In [11]:
a.fullmul(b)

TaylorModel(Poly([0, 1, 4, 10, 20, 30, 34, 31, 20], 8), interval([-13.0, 37.0]), interval([-1.0, 1.0]))

In [12]:
a.fullmul(b).truncation(7)

TaylorModel(Poly([0, 1, 4, 10, 20, 30, 34, 31], 7), interval([-13.0, 57.0]), interval([-1.0, 1.0]))

In [13]:
a*b

TaylorModel(Poly([0, 1, 4, 10, 20], 4), interval([-74.0, 152.0]), interval([-1.0, 1.0]))

In [13]:
2*a

TaylorModel(Poly([0, 2, 4, 6, 8], 4), interval([0.0, 2.0]), interval([-1.0, 1.0]))

In [14]:
a/2

TaylorModel(Poly([0.0, 0.5, 1.0, 1.5, 2.0], 4), interval([0.0, 0.5]), interval([-1.0, 1.0]))

In [15]:
Poly.one(5)

Poly([1, 0, 0, 0, 0, 0], 5)

In [16]:
a**5

TaylorModel(Poly([0, 0, 0, 0, 0], 4), interval([-78764.0, 161051.0]), interval([-1.0, 1.0]))

In [17]:
imath.exp(1)

interval([2.718281828459045, 2.7182818284590455])

In [18]:
math.factorial(2)

2

In [19]:
imath.exp(0)/math.factorial(4)

interval([0.041666666666666664, 0.04166666666666667])

In [20]:
Taylor.element('exp',4,0,interval([-1.0, 1.0]))

TaylorModel(Poly([1.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664], 4), interval([-0.022652348570492052, 0.022652348570492052]), interval([-1.0, 1.0]))

In [21]:
Taylor.element('sin',4,0,interval([-1.0, 1.0]))

TaylorModel(Poly([0.0, 1.0, -0.0, -0.16666666666666666, 0.0], 4), interval([-0.008333333333333335, 0.008333333333333335]), interval([-1.0, 1.0]))

In [22]:
Taylor.element('cos',4,0,interval([-1.0, 1.0]))

TaylorModel(Poly([1.0, -0.0, -0.5, 0.0, 0.041666666666666664], 4), interval([-0.0070122582067324735, 0.0070122582067324735]), interval([-1.0, 1.0]))

In [24]:
a=Taylor(Poly([0,1,2,3,4],4),interval([0,1]),interval([-1,1]))

In [25]:
a.bound()

interval([-4.0, 11.0])

In [26]:
Mg=Taylor.element('sin',4,0,interval([-4.0, 11.0]))
Mg

TaylorModel(Poly([0.0, 1.0, -0.0, -0.16666666666666666, 0.0], 4), interval([-1342.0916666666672, 1342.0916666666672]), interval([-4.0, 11.0]))

In [28]:
a

TaylorModel(Poly([0, 1, 2, 3, 4], 4), interval([0.0, 1.0]), interval([-1.0, 1.0]))

In [29]:
M1=a

In [30]:
Mg.P

Poly([0.0, 1.0, -0.0, -0.16666666666666666, 0.0], 4)

In [31]:
M=M1.compo_poly(Mg.P,'best')
M

TaylorModel(Poly([0.0, 1.0, 2.0, 2.8333333333333335, 3.0], 4), interval([-220.66666666666666, 108.16666666666667]), interval([-1.0, 1.0]))

In [32]:
Taylor(M.P,M.I+Mg.I,interval([-1,1]))

TaylorModel(Poly([0.0, 1.0, 2.0, 2.8333333333333335, 3.0], 4), interval([-1562.758333333334, 1450.258333333334]), interval([-1.0, 1.0]))

In [33]:
a.compose('sin',4,0)

TaylorModel(Poly([0.0, 1.0, 2.0, 2.8333333333333335, 3.0], 4), interval([-1562.758333333334, 1450.258333333334]), interval([-1.0, 1.0]))